# Arabic Auto-Complete System

## Imports

In [26]:
import os  
import re   
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re
import numpy as np

## Text Cleaning Function

In [5]:
import re

def clean_arabic_text(text):
    # 1. Remove tashkeel including tatweel 
    text = re.sub(r'[ً-ْـ]', '', text)
    
    # 2. Remove punctuation and special characters (keeping Arabic letters, numbers, and basic punctuation)
    text = re.sub(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s٠-٩۰-۹.,،؛؟!]', ' ', text)

    # 3. Normalize Arabic characters
    text = re.sub(r'[آأإ]', 'ا', text)  
    text = re.sub(r'ى', 'ي', text)     
    text = re.sub(r'ة', 'ه', text)     

    # 4. Remove English letters and Western digits
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'[0-9]', '', text)

    # 5. Normalize Arabic punctuation to standard ones
    text = text.replace('؟', '?')  # Arabic question mark
    text = text.replace('،', ',')  # Arabic comma
    text = text.replace('؛', ';')  # Arabic semicolon

    # 6. Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text).strip()

    # 7. Remove repeated characters 
    text = re.sub(r'(.)\1+', r'\1', text)

    return text


## Data Loading and Cleaning

In [7]:
file_names = ['file1.txt', 'file2.txt', 'file3.txt', 'file4.txt']  # data files
all_clean_lines = []  

for file in file_names:
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()  
        for line in lines:
            if 'NULL' in line:
                continue  # skip lines with the word NULL
            clean_line = clean_arabic_text(line)
            if clean_line:  # don't save empty lines
                all_clean_lines.append(clean_line)


##  Save Cleaned Data


In [9]:
with open('cleaned_data.txt', 'w', encoding='utf-8') as f:
    for line in all_clean_lines:
        f.write(line + '\n')

## Read the data

In [11]:
with open('cleaned_data.txt', 'r', encoding='utf-8') as file:
    texts = file.readlines()

## Info about the data

In [13]:
print(f"Total texts: {len(texts)}")
print("Sample texts:")
for i in range(3):
    print(f"{i+1}. {texts[i].strip()}")

Total texts: 7904
Sample texts:
1. اهنئ الدكتور احمد جمال الدين, القيادي بحزب مصر, بمناسبه صدور اولي روايه
2. امير عيد هو الي فعلا يتقال عليه ستريكر صريح
3. الصداقه تزرع الحياه ازهارا


## Tokenization and Sequence Preparation

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1
print(f"\nTotal unique words: {total_words}")
input_sequences = []
for text in texts:
    token_list = tokenizer.texts_to_sequences([text])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


Total unique words: 7868


## Padding Sequences

In [17]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
print("\nSample padded sequences:")
print(input_sequences[:3])



Sample padded sequences:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 3117  378]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0 3117  378  547]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0 3117  378  547  255]]


## Split Features (X) and Labels (y)

In [19]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)


##  Build and Train the Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Build the model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=64, input_length=max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X, y, epochs=30, verbose=1)



C:\Users\husse\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.0267 - loss: 8.2072
Epoch 2/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.0363 - loss: 7.1165
Epoch 3/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.0683 - loss: 6.1897
Epoch 4/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 36s 33ms/step - accuracy: 0.1356 - loss: 5.2628
Epoch 5/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.2409 - loss: 4.3463
Epoch 6/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 36s 33ms/step - accuracy: 0.3694 - loss: 3.5682
Epoch 7/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - accuracy: 0.4971 - loss: 2.8459
Epoch 8/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - accuracy: 0.5956 - loss: 2.2733
Epoch 9/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.6783 - loss: 1.8147
Epoch 10/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.7497 - loss: 1.4374
Epoch 11/30
1097/1097 ━━━━━━━━━━━━━━━━━━━━ 35s 32ms/step - accuracy: 0.7935 - loss: 1.1621
Epoch 12

In [56]:
def predict_sequence(model, tokenizer, start_word, next_words, max_len):
    result = start_word
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                result += " " + word
                break
    return result

# Try an example
start_word = "دي"
next_words = 9
generated_text = predict_sequence(model, tokenizer, start_word, next_words, max_sequence_len)
print("النص الناتج:", generated_text)

النص الناتج: دي اخلاق السلف الصالح والسلام الامور دليل علي ضعف المستوي
